In [ ]:
import networkx as nx
from braket.aws import AwsDevice
from braket.circuits import Circuit
from braket.devices import Devices
import time
import matplotlib.pyplot as plt
%matplotlib inline

import src.graphconnector as gc

In [ ]:
# Ankaa3_device = AwsDevice("arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-3")
Emerald_device = AwsDevice("arn:aws:braket:eu-north-1::device/qpu/iqm/Emerald")

G = Emerald_device.topology_graph

start_time = time.time()
print("Initializing solver...")
total_connectivity = Emerald_device.properties.paradigm.connectivity.connectivityGraph
qubits = [qub for qub in total_connectivity]
qubits.sort(key=float)

final_sets, solver = gc.generate_circuits(total_connectivity, max_sets=4, compaction = True, redundancy = True)

end_time = time.time()

print(f"\nDONE in {end_time - start_time:.4f} seconds.")
print(f"Total Edges in Graph: {len(solver.all_edges)}")
print("-" * 30)


for i in range(len(final_sets)):
    
    red_edges = list(final_sets[i])
    connected_qubits = [str(qub_pair[0]) for qub_pair in red_edges] + [str(qub_pair[1]) for qub_pair in red_edges]
    connected_qubits.sort(key=float)
    # print("Connected Qubits:\n",connected_qubits,"\n")
    # print("Total Qubits:\n",qubits,"\n")
    print("Efficiency:", len(connected_qubits)/len(qubits))
    
    # 3. Create a color map based on G.edges()
    edge_colors = []
    widths = []
    for u, v in G.edges():
        u = str(u)
        v = str(v)
        # Check if the edge (or its reverse) is in your red_subset
        if (u, v) in red_edges or (v, u) in red_edges:
            edge_colors.append('red')
            widths.append(6)
        else:
            edge_colors.append('black')
            widths.append(3)
    
    black_edges = [e for e in G.edges() if e not in red_edges and (e[1], e[0]) not in red_edges]

    if i == 0:
        pos = nx.forceatlas2_layout(G)
    nx.draw(G, pos, with_labels=True, edge_color=edge_colors, node_color = 'black' ,font_color="white", width = widths, arrows=False)
    plt.show()

